In [1]:
import pandas as pd
from io import StringIO
import os
from datetime import datetime, timedelta
from dataclasses import dataclass, field
from typing import List, Dict, Optional, Union, Set, Any
from enum import Enum
import pickle
import pandas as pd
import statsmodels.api as sm
import numpy as np

from nst_scraper import nst_on_ice_scraper
from db_utils import update_player_db, check_last_update, append_player_ids, get_player_full_name
from team_utils import get_most_recent_game_id
from game_utils import get_game_boxscore, display_boxscore

from dotenv import load_dotenv

pd.set_option('display.max_columns', None)
np.set_printoptions(legacy='1.25')

In [2]:
player_stats_df = nst_on_ice_scraper(fromseason=20242025, thruseason=20242025, startdate='', enddate='', rate='y')
player_stats_df.head()

,player,team,position,gp,toi,toi/gp,goals/60,total_assists/60,first_assists/60,second_assists/60,total_points/60,ipp,shots/60,sh%,ixg/60,icf/60,iff/60,iscf/60,ihdcf/60,rush_attempts/60,rebounds_created/60,pim/60,total_penalties/60,minor/60,major/60,misconduct/60,penalties_drawn/60,giveaways/60,takeaways/60,hits/60,hits_taken/60,shots_blocked/60,faceoffs_won/60,faceoffs_lost/60,faceoffs_%
0,Ryan Suter,STL,D,27,530.150000,19.635185,0.11,0.45,0.11,0.34,0.57,20.83,2.60,4.35,0.11,7.02,3.96,1.92,0.23,0.34,0.57,1.58,0.79,0.79,0.00,0.0,0.11,3.06,0.57,1.58,2.94,3.51,0.00,0.00,-
1,Brent Burns,CAR,D,26,427.966667,16.460256,0.14,0.84,0.42,0.42,0.98,36.84,5.47,2.56,0.22,14.86,8.97,3.93,0.28,0.56,1.12,0.56,0.28,0.28,0.00,0.0,0.28,4.77,1.54,0.42,2.24,2.94,0.00,0.00,-
2,Corey Perry,EDM,R,26,262.566667,10.098718,0.91,0.00,0.00,0.00,0.91,50.00,4.80,19.05,0.67,11.20,7.77,7.08,3.66,0.00,0.46,3.66,1.14,0.69,0.46,0.0,1.83,1.83,0.91,1.83,4.34,1.14,0.46,0.46,11.43
3,Alex Ovechkin,WSH,L,18,215.250000,11.958333,2.51,1.95,1.11,0.84,4.46,72.73,9.48,26.47,0.91,19.23,13.66,11.15,4.46,0.00,1.39,1.67,0.84,0.84,0.00,0.0,0.00,1.95,1.11,11.43,2.23,0.84,0.00,0.00,-
4,Evgeni Malkin,PIT,C,28,409.850000,14.637500,0.59,1.46,1.02,0.44,2.05,77.78,5.42,10.81,0.77,10.69,7.03,6.29,3.22,0.00,1.32,1.46,0.73,0.73,0.00,0.0,0.88,4.10,1.02,1.90,3.81,2.64,12.59,17.57,6.11


In [3]:
goalie_stats_df = nst_on_ice_scraper(fromseason=20242025, thruseason=20242025, startdate='', enddate='', pos='g', rate='y', stdoi='g')
goalie_stats_df.head()

,player,team,gp,toi,toi/gp,shots_against/60,saves/60,sv%,gaa,gsaa/60,xg_against/60,hd_shots_against/60,hd_saves/60,hdsv%,hdgaa,hdgsaa/60,md_shots_against/60,md_saves/60,mdsv%,mdgaa,mdgsaa/60,ld_shots_against/60,ld_saves/60,ldsv%,ldgaa,ldgsaa/60,rush_attempts_against/60,rebound_attempts_against/60,avg._shot_distance,avg._goal_distance
0,Marc-Andre Fleury,MIN,6,299.833333,49.972222,27.62,25.61,0.928,2.00,0.43,1.70,4.40,3.60,0.818,0.80,-0.01,6.00,5.00,0.833,1.00,-0.29,15.41,15.41,1.000,0.00,0.50,1.80,3.40,39.61,30.40
1,Jonathan Quick,NYR,7,319.583333,45.654762,31.17,29.66,0.952,1.50,1.25,2.60,8.26,7.89,0.955,0.38,1.10,8.64,7.70,0.891,0.94,0.09,13.52,13.33,0.986,0.19,0.25,2.44,5.82,33.71,23.38
2,James Reimer,"ANA, BUF",4,195.983333,48.995833,27.25,24.80,0.910,2.45,-0.05,2.66,3.37,1.84,0.545,1.53,-0.93,12.55,11.63,0.927,0.92,0.58,11.02,11.02,1.000,0.00,0.36,1.84,5.20,32.03,16.88
3,Semyon Varlamov,NYI,10,499.450000,49.945000,25.35,23.07,0.910,2.28,-0.05,2.36,7.33,6.37,0.869,0.96,0.35,6.01,5.29,0.880,0.72,-0.01,9.85,9.25,0.939,0.60,-0.28,1.92,4.44,35.72,20.05
4,Jacob Markstrom,N.J,20,966.650000,48.332500,26.07,23.65,0.907,2.42,-0.12,2.19,5.77,4.53,0.785,1.24,-0.21,7.20,6.39,0.888,0.81,0.05,11.11,10.74,0.966,0.37,-0.01,3.41,4.84,40.53,21.26


In [4]:
class Position(Enum):
    C = 'C'
    L = 'L'
    R = 'R'
    D = 'D'
    G = 'G'
    
    @property
    def category(self) -> str:
        if self in {Position.C, Position.L, Position.R}:
            return 'F'
        elif self == Position.D:
            return 'D'
        elif self == Position.G:
            return 'G'
    
    def __str__(self) -> str:
        return self.value

In [5]:
@dataclass
class Player:
    name: str
    team: str
    position: Position
    player_id: Optional[int] = None

    def __str__(self) -> str:
        """
        Returns a string representation of the player.
        """
        return f"{self.name} ({self.position}) - {self.team}"

    def to_dict(self) -> Dict[str, Optional[str]]:
        """
        Converts the Player instance into a dictionary.
        
        Returns:
            Dict[str, Optional[str]]: A dictionary representation of the player.
        """
        return {
            'player_id': self.player_id,
            'name': self.name,
            'team': self.team,
            'position': self.position.value
        }

In [6]:
@dataclass
class Lineup:
    name: str
    forwards: List[Optional[Player]] = field(default_factory=lambda: [None] * 12)
    defense: List[Optional[Player]] = field(default_factory=lambda: [None] * 6)
    goalies: List[Optional[Player]] = field(default_factory=lambda: [None] * 2)
    
    ALLOWED_FORWARD_CATEGORIES = {'F'}
    ALLOWED_DEFENSE_CATEGORY = 'D'
    ALLOWED_GOALIE_CATEGORY = 'G'
    
    def __post_init__(self):
        self.validate_lineup_size()
    
    def validate_lineup_size(self):
        total_players = sum(player is not None for player in self.forwards + self.defense + self.goalies)
        if total_players > 20:
            raise ValueError(f"Total number of players ({total_players}) exceeds the hard limit of 20.")
    
    def add_player(
        self,
        category: str,
        player: Player,
        slot: int,
        allowed_categories: Union[str, Set[str]],
        max_slots: int
    ):
        """
        Adds a player to the specified category and slot after validating their position category.
        Ensures that the total number of players does not exceed 20.
        """
        if isinstance(allowed_categories, str):
            allowed_categories = {allowed_categories}
        elif isinstance(allowed_categories, set):
            allowed_categories = allowed_categories
        else:
            raise TypeError("allowed_categories must be a string or a set of strings.")
        
        if player.position.category not in allowed_categories:
            raise ValueError(
                f"Cannot add player '{player.name}' with position '{player.position.value}' "
                f"to {category}. Allowed categories: {', '.join(allowed_categories)}."
            )
        
        if not 0 <= slot < max_slots:
            raise IndexError(f"{category.capitalize()} slot must be between 0 and {max_slots - 1}.")
        
        current_category = getattr(self, category)
        if current_category[slot]:
            existing_player = current_category[slot].name
            print(f"Warning: Slot {slot + 1} in {category} is already occupied by '{existing_player}'. Overwriting.")
        
        # Check total players before adding
        total_players = sum(player is not None for player in self.forwards + self.defense + self.goalies)
        if current_category[slot] is None and total_players >= 20:
            raise ValueError("Cannot add more players. The lineup has reached the hard limit of 20 players.")
        
        current_category[slot] = player
        setattr(self, category, current_category)
        print(f"Added player '{player.name}' to {category.capitalize()} slot {slot + 1}.")
    
    def add_forward(self, player: Player, slot: int):
        self.add_player(
            category='forwards',
            player=player,
            slot=slot,
            allowed_categories=self.ALLOWED_FORWARD_CATEGORIES,
            max_slots=len(self.forwards)
        )
    
    def add_defense(self, player: Player, slot: int):
        self.add_player(
            category='defense',
            player=player,
            slot=slot,
            allowed_categories={self.ALLOWED_DEFENSE_CATEGORY},
            max_slots=len(self.defense)
        )
    
    def set_goalie(self, player: Player, slot: int):
        self.add_player(
            category='goalies',
            player=player,
            slot=slot,
            allowed_categories={self.ALLOWED_GOALIE_CATEGORY},
            max_slots=len(self.goalies)
        )
    
    def adjust_slots(self, category: str, delta: int):
        """
        Adjusts the number of slots in the specified category by delta.
        Allows ±1 adjustment only.
        
        Args:
            category (str): The category to adjust ('forwards' or 'defense').
            delta (int): The change in number of slots (+1 or -1).
        """
        if category not in {'forwards', 'defense'}:
            raise ValueError("Can only adjust 'forwards' or 'defense' categories.")
        if delta not in {-1, 1}:
            raise ValueError("Delta must be either +1 or -1.")
        
        current_slots = getattr(self, category)
        new_slot_count = len(current_slots) + delta
        
        if category == 'forwards':
            if not (11 <= new_slot_count <= 13):
                raise ValueError("Number of forwards can only vary by ±1 from the default of 12.")
        elif category == 'defense':
            if not (5 <= new_slot_count <= 7):
                raise ValueError("Number of defensemen can only vary by ±1 from the default of 6.")
        
        if delta == 1:
            current_slots.append(None)
        elif delta == -1:
            removed_player = current_slots.pop()
            if removed_player:
                print(f"Removed player '{removed_player.name}' from {category}.")
        
        setattr(self, category, current_slots)
        print(f"Adjusted {category} slots to {len(getattr(self, category))}.")
        self.validate_lineup_size()
    
    def display_lineup(self):
        """
        Prints the current lineup in a structured format.
        """
        print(f"Lineup: {self.name}\n")
        
        for category, title in [('forwards', 'Forwards'), ('defense', 'Defense'), ('goalies', 'Goalies')]:
            print(f"{title}:")
            for idx, player in enumerate(getattr(self, category), start=1):
                player_info = str(player) if player else 'Empty'
                print(f"  Slot {idx}: {player_info}")
            print()
    
    def to_dataframe(self) -> pd.DataFrame:
        """
        Converts the lineup into a pandas DataFrame.
        Conditionally includes player attributes if they are present.
        """
        data = []
        for category, pos in [('forwards', 'f'), ('defense', 'd'), ('goalies', 'g')]:
            for idx, player in enumerate(getattr(self, category), start=1):
                player_dict = {
                    'Position': f"{pos}{idx}",
                    'Player': player.name if player else 'Empty'
                }
                # Conditionally add 'player_id' if it exists
                if player and player.player_id is not None:
                    player_dict['Player ID'] = player.player_id
                data.append(player_dict)
        
        df = pd.DataFrame(data)
        
        # Optionally, remove columns where all values are NaN
        df.dropna(axis=1, how='all', inplace=True)
        
        return df
    
    def to_transposed_dataframe(self) -> pd.DataFrame:
        """
        Transposes the lineup DataFrame so that each column represents a position-slot combination
        and the row contains the corresponding player names.
        Conditionally includes additional player attributes if they are present.
        """
        df = self.to_dataframe()
        
        # Initialize dictionaries to hold player names and optional IDs
        player_data = {}
        id_data = {}
        
        for _, row in df.iterrows():
            pos = row['Position']
            player_name = row['Player']
            player_data[pos] = player_name
            
            # Handle 'Player ID' if it exists
            if 'Player ID' in row and pd.notna(row['Player ID']):
                id_data[f"{pos}_ID"] = row['Player ID']
        
        # Combine player names and IDs into a single dictionary
        transposed_data = {**player_data, **id_data}
        
        # Create the transposed DataFrame with a single row
        transposed_df = pd.DataFrame([transposed_data])
        
        return transposed_df

In [7]:
# Creating Player instances from the player_stats_df DataFrame
player_list = []
for _, row in player_stats_df.iterrows():
    try:
        # !! Take only first position if multiple are listed, this is a hack to deal with the fact that some players have multiple positions listed
        position = row['position'].split(',')[0].strip()
        position_enum = Position(position)  # Convert abbreviation to Position Enum
    except ValueError:
        print(f"Invalid position '{row['position']}' for player '{row['player']}'. Skipping.")
        continue
    
    player = Player(
        name=row['player'],
        team=row['team'],
        position=position_enum
        # player_id is not set initially
    )
    player_list.append(player)

# Convert player list to DataFrame to verify
pd.DataFrame([{
    'name': player.name,
    'team': player.team, 
    'position': player.position.value
} for player in player_list]).head()

,name,team,position
0,Ryan Suter,STL,D
1,Brent Burns,CAR,D
2,Corey Perry,EDM,R
3,Alex Ovechkin,WSH,L
4,Evgeni Malkin,PIT,C


In [8]:
goalie_stats_df.head()

,player,team,gp,toi,toi/gp,shots_against/60,saves/60,sv%,gaa,gsaa/60,xg_against/60,hd_shots_against/60,hd_saves/60,hdsv%,hdgaa,hdgsaa/60,md_shots_against/60,md_saves/60,mdsv%,mdgaa,mdgsaa/60,ld_shots_against/60,ld_saves/60,ldsv%,ldgaa,ldgsaa/60,rush_attempts_against/60,rebound_attempts_against/60,avg._shot_distance,avg._goal_distance
0,Marc-Andre Fleury,MIN,6,299.833333,49.972222,27.62,25.61,0.928,2.00,0.43,1.70,4.40,3.60,0.818,0.80,-0.01,6.00,5.00,0.833,1.00,-0.29,15.41,15.41,1.000,0.00,0.50,1.80,3.40,39.61,30.40
1,Jonathan Quick,NYR,7,319.583333,45.654762,31.17,29.66,0.952,1.50,1.25,2.60,8.26,7.89,0.955,0.38,1.10,8.64,7.70,0.891,0.94,0.09,13.52,13.33,0.986,0.19,0.25,2.44,5.82,33.71,23.38
2,James Reimer,"ANA, BUF",4,195.983333,48.995833,27.25,24.80,0.910,2.45,-0.05,2.66,3.37,1.84,0.545,1.53,-0.93,12.55,11.63,0.927,0.92,0.58,11.02,11.02,1.000,0.00,0.36,1.84,5.20,32.03,16.88
3,Semyon Varlamov,NYI,10,499.450000,49.945000,25.35,23.07,0.910,2.28,-0.05,2.36,7.33,6.37,0.869,0.96,0.35,6.01,5.29,0.880,0.72,-0.01,9.85,9.25,0.939,0.60,-0.28,1.92,4.44,35.72,20.05
4,Jacob Markstrom,N.J,20,966.650000,48.332500,26.07,23.65,0.907,2.42,-0.12,2.19,5.77,4.53,0.785,1.24,-0.21,7.20,6.39,0.888,0.81,0.05,11.11,10.74,0.966,0.37,-0.01,3.41,4.84,40.53,21.26


In [9]:
# Creating Player instances from the goalie_stats_df DataFrame
goalie_list = []
for _, row in goalie_stats_df.iterrows():
    player = Player(
        name=row['player'],
        team=row['team'],
        position=Position.G  # Set position to 'G' for goalies
        # player_id is not set initially
    )
    goalie_list.append(player)

# Convert goalie list to DataFrame to verify
pd.DataFrame([{
    'name': player.name,
    'team': player.team, 
    'position': player.position.value
} for player in goalie_list]).head()

,name,team,position
0,Marc-Andre Fleury,MIN,G
1,Jonathan Quick,NYR,G
2,James Reimer,"ANA, BUF",G
3,Semyon Varlamov,NYI,G
4,Jacob Markstrom,N.J,G


In [10]:
today_datetime= datetime.now()
yesterday_datetime = today_datetime - timedelta(days=1, hours=6) # UTC offset
yesterday = yesterday_datetime.strftime('%Y-%m-%d')
yesterday

'2024-12-06'

In [11]:
# Load environment variables from .env file
load_dotenv()

# Construct the database configuration dictionary
db_config = {
    'dbname': os.getenv('DB_NAME'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'host': os.getenv('DB_HOST'),
    'port': os.getenv('DB_PORT')
}

In [12]:
# Check the last update time of the players database
last_update = check_last_update(db_config)

INFO:db_utils:Database connection established.
INFO:db_utils:Last database update was on: 2024-12-06
INFO:db_utils:Database connection closed.


In [13]:
# Convert last_update to datetime
last_update_dt = datetime.strptime(last_update, '%Y-%m-%d')
today_dt = datetime.strptime(today_datetime.strftime('%Y-%m-%d'), '%Y-%m-%d')
yesterday_dt = datetime.strptime(yesterday, '%Y-%m-%d')

# Only update if last update was before yesterday
if last_update_dt not in [today_dt, yesterday_dt]:
    # Update the player database from last update to yesterday
    update_player_db(last_update, yesterday, db_config, skip_existing=True)
else:
    print(f"No need to update the player database. Last update was on: {last_update}")


No need to update the player database. Last update was on: 2024-12-06


In [14]:
def add_player_to_lineup(lineup: Lineup, player: Player, category: str):
    """
    Adds a player to the lineup in the specified category, handling potential errors.
    
    Args:
        lineup (Lineup): The lineup object.
        player (Player): The player to add.
        category (str): The category ('forwards', 'defense', 'goalies').
    """
    try:
        if category == 'forwards':
            slot = next(i for i, p in enumerate(lineup.forwards) if p is None)
            lineup.add_forward(player, slot)
        elif category == 'defense':
            slot = next(i for i, p in enumerate(lineup.defense) if p is None)
            lineup.add_defense(player, slot)
        elif category == 'goalies':
            slot = next(i for i, p in enumerate(lineup.goalies) if p is None)
            lineup.set_goalie(player, slot)
        else:
            print(f"Unknown category '{category}'.")
    except StopIteration:
        print(f"No available slots to add player '{player.name}' in category '{category}'.")
    except ValueError as ve:
        print(ve)
    except IndexError as ie:
        print(ie)

In [15]:
# TODO this function just creates a lineup from the player_list and goalie_list
def create_lineup(team) -> Lineup:
    """
    Creates and displays a lineup consisting of players from the specified team.
    
    Args:
        team (str): The team name to filter players.
    """
    # Creating two lineup objects
    lineup1 = Lineup("Lineup 1")
    
    # Adding forwards to lineup1
    forward_count = 0
    for player in player_list:
        if player.team == team:
            try:
                lineup1.add_forward(player, forward_count)
                forward_count += 1
                if forward_count >= 12:
                    break
            except ValueError as e:
                print(f"Skipping player '{player.name}': {e}")
            except IndexError as e:
                print(f"Skipping player '{player.name}': {e}")
        else:
            continue  # Proceed to the next player if not in the specified team
    
    # Adding defense to lineup1
    defense_count = 0
    for player in player_list:
        if player.team == team:
            try:
                lineup1.add_defense(player, defense_count)
                defense_count += 1
                if defense_count >= 6:
                    break
            except ValueError as e:
                print(f"Skipping player '{player.name}': {e}")
            except IndexError as e:
                print(f"Skipping player '{player.name}': {e}")
        else:
            continue  # Proceed to the next player if not in the specified team
    
    # Adding goalies to lineup1
    goalie_count = 0
    for goalie in goalie_list:
        if goalie.team != team:
            continue  # Proceed to the next goalie if not in the specified team
        if goalie_count >= 2:
            print("Maximum of two goalies have been assigned.")
            break
        try:
            lineup1.set_goalie(goalie, goalie_count)
            goalie_count += 1
        except ValueError as e:
            print(f"Skipping goalie '{goalie.name}': {e}")
        except IndexError as e:
            print(f"Skipping goalie '{goalie.name}': {e}")
    
    # Display the lineup
    # lineup1.display_lineup()
    return lineup1

my_lineup = create_lineup('TOR')

Added player 'Ryan Reaves' to Forwards slot 1.
Added player 'Max Pacioretty' to Forwards slot 2.
Added player 'John Tavares' to Forwards slot 3.
Skipping player 'Oliver Ekman-Larsson': Cannot add player 'Oliver Ekman-Larsson' with position 'D' to forwards. Allowed categories: F.
Skipping player 'Chris Tanev': Cannot add player 'Chris Tanev' with position 'D' to forwards. Allowed categories: F.
Skipping player 'Jani Hakanpää': Cannot add player 'Jani Hakanpää' with position 'D' to forwards. Allowed categories: F.
Skipping player 'Morgan Rielly': Cannot add player 'Morgan Rielly' with position 'D' to forwards. Allowed categories: F.
Skipping player 'Jake McCabe': Cannot add player 'Jake McCabe' with position 'D' to forwards. Allowed categories: F.
Added player 'Max Domi' to Forwards slot 4.
Added player 'William Nylander' to Forwards slot 5.
Added player 'Mitch Marner' to Forwards slot 6.
Added player 'Steven Lorentz' to Forwards slot 7.
Skipping player 'Philippe Myers': Cannot add playe

In [16]:
def create_flexible_lineup(team: str) -> Lineup:
    """
    Creates a flexible lineup for the specified team, allowing for ±1 forward or defense spot.
    
    Args:
        team (str): The team name to filter players.
    
    Returns:
        Lineup: The configured lineup object.
    """
    lineup = Lineup(f"Flexible Lineup for {team}")
    
    # Example logic to adjust slots based on team strategy
    # Here, we simply allow flexibility; implement specific rules as needed
    allow_extra_forward = True  # Example condition
    allow_extra_defense = False  # Example condition
    
    if allow_extra_forward:
        lineup.adjust_slots('forwards', 1)  # Increase forwards to 13
    elif allow_extra_defense:
        lineup.adjust_slots('defense', 1)  # Increase defense to 7
    
    # Add players to the lineup
    forward_count = 0
    defense_count = 0
    goalie_count = 0
    
    for player in player_list:
        if player.team == team:
            if player.position.category == 'F' and forward_count < len(lineup.forwards):
                try:
                    lineup.add_forward(player, forward_count)
                    forward_count += 1
                except ValueError:
                    continue
            elif player.position.category == 'D' and defense_count < len(lineup.defense):
                try:
                    lineup.add_defense(player, defense_count)
                    defense_count += 1
                except ValueError:
                    continue
    
    for goalie in goalie_list:
        if goalie.team == team and goalie_count < len(lineup.goalies):
            try:
                lineup.set_goalie(goalie, goalie_count)
                goalie_count += 1
            except ValueError:
                continue
    
    return lineup

# Create and display a flexible lineup for 'TOR'
# flexible_lineup = create_flexible_lineup('TOR')
# flexible_lineup.display_lineup()

In [17]:
my_lineup.to_dataframe()

,Position,Player
0,f1,Ryan Reaves
1,f2,Max Pacioretty
2,f3,John Tavares
3,f4,Max Domi
4,f5,William Nylander
5,f6,Mitch Marner
6,f7,Steven Lorentz
7,f8,Auston Matthews
8,f9,Alex Nylander
9,f10,David Kampf


In [18]:
# Convert the lineup to a transposed DataFrame
transposed_lineup_df = my_lineup.to_transposed_dataframe()

# Display the transposed DataFrame
transposed_lineup_df

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,d1,d2,d3,d4,d5,d6,g1,g2
0,Ryan Reaves,Max Pacioretty,John Tavares,Max Domi,William Nylander,Mitch Marner,Steven Lorentz,Auston Matthews,Alex Nylander,David Kampf,Connor Dewar,Pontus Holmberg,Oliver Ekman-Larsson,Chris Tanev,Jani Hakanpää,Morgan Rielly,Jake McCabe,Philippe Myers,Anthony Stolarz,Joseph Woll


In [19]:
def assign_player_ids_to_lineup(transposed_lineup_df, db_config):
    """
    Processes the transposed_lineup_df DataFrame to append player IDs next to each player's name.
    
    This function performs the following steps:
        1. Extracts all unique player names from the lineup DataFrame.
        2. Creates Player instances for each player name.
        3. Uses the append_player_ids function to assign player IDs to each Player object.
        4. Inserts new columns into the DataFrame with the corresponding player IDs.
    
    Args:
        transposed_lineup_df (pd.DataFrame): The transposed lineup DataFrame with player names.
        db_config (dict): Database configuration with keys: dbname, user, password, host, port.
    
    Returns:
        pd.DataFrame: The updated DataFrame with player IDs added.
    """
    # Ensure the DataFrame has only one row
    if transposed_lineup_df.shape[0] != 1:
        raise ValueError("transposed_lineup_df should contain exactly one row representing the lineup.")

    # Extract player names from the DataFrame
    player_columns = transposed_lineup_df.columns
    player_names = transposed_lineup_df.iloc[0].tolist()

    # Create Player instances
    player_list = []
    for name in player_names:
        if name != 'Empty':
            player = Player(name=name, team=None, position=None)  # Team and position can be set if available
            player_list.append(player)

    # Append player IDs using the existing function
    append_player_ids(player_list, db_config)

    # Create a mapping from player name to player_id
    name_to_id = {player.name: player.player_id for player in player_list if player.player_id is not None}

    # Initialize a copy of the DataFrame to avoid modifying the original
    updated_df = transposed_lineup_df.copy()

    # Insert player_id columns adjacent to each player name column
    for col in player_columns:
        player_name = updated_df.at[0, col]
        if player_name != 'Empty':
            player_id = name_to_id.get(player_name, None)
            id_column = f"{col}_id"
            updated_df[id_column] = player_id
        else:
            id_column = f"{col}_id"
            updated_df[id_column] = None  # Assign None for empty slots

    return updated_df

In [20]:
# Assuming you have already created `transposed_lineup_df` and `db_config`
updated_lineup_df = assign_player_ids_to_lineup(transposed_lineup_df, db_config)

# Display the updated DataFrame
updated_lineup_df

INFO:db_utils:Database connection established.
INFO:db_utils:Assigned player_id 8471817 to Ryan Reaves.
INFO:db_utils:Assigned player_id 8474157 to Max Pacioretty.
INFO:db_utils:Assigned player_id 8475166 to John Tavares.
INFO:db_utils:Assigned player_id 8477503 to Max Domi.
INFO:db_utils:Assigned player_id 8477939 to William Nylander.
INFO:db_utils:Assigned player_id 8478483 to Mitch Marner.
INFO:db_utils:Assigned player_id 8478904 to Steven Lorentz.
INFO:db_utils:Assigned player_id 8479318 to Auston Matthews.
INFO:db_utils:Assigned player_id 8479423 to Alex Nylander.
INFO:db_utils:Assigned player_id 8480144 to David Kampf.
INFO:db_utils:Assigned player_id 8480980 to Connor Dewar.
INFO:db_utils:Assigned player_id 8480995 to Pontus Holmberg.
INFO:db_utils:Assigned player_id 8475171 to Oliver Ekman-Larsson.
INFO:db_utils:Assigned player_id 8475690 to Chris Tanev.
INFO:db_utils:Assigned player_id 8475825 to Jani Hakanpää.
INFO:db_utils:Assigned player_id 8476853 to Morgan Rielly.
INFO:db

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,d1,d2,d3,d4,d5,d6,g1,g2,f1_id,f2_id,f3_id,f4_id,f5_id,f6_id,f7_id,f8_id,f9_id,f10_id,f11_id,f12_id,d1_id,d2_id,d3_id,d4_id,d5_id,d6_id,g1_id,g2_id
0,Ryan Reaves,Max Pacioretty,John Tavares,Max Domi,William Nylander,Mitch Marner,Steven Lorentz,Auston Matthews,Alex Nylander,David Kampf,Connor Dewar,Pontus Holmberg,Oliver Ekman-Larsson,Chris Tanev,Jani Hakanpää,Morgan Rielly,Jake McCabe,Philippe Myers,Anthony Stolarz,Joseph Woll,8471817,8474157,8475166,8477503,8477939,8478483,8478904,8479318,8479423,8480144,8480980,8480995,8475171,8475690,8475825,8476853,8476931,8479026,8476932,8479361


In [21]:
# player_stats_df = nst_on_ice_scraper(fromseason=20242025, thruseason=20242025, startdate='', enddate='')
player_stats_df.head() 

,player,team,position,gp,toi,toi/gp,goals/60,total_assists/60,first_assists/60,second_assists/60,total_points/60,ipp,shots/60,sh%,ixg/60,icf/60,iff/60,iscf/60,ihdcf/60,rush_attempts/60,rebounds_created/60,pim/60,total_penalties/60,minor/60,major/60,misconduct/60,penalties_drawn/60,giveaways/60,takeaways/60,hits/60,hits_taken/60,shots_blocked/60,faceoffs_won/60,faceoffs_lost/60,faceoffs_%
0,Ryan Suter,STL,D,27,530.150000,19.635185,0.11,0.45,0.11,0.34,0.57,20.83,2.60,4.35,0.11,7.02,3.96,1.92,0.23,0.34,0.57,1.58,0.79,0.79,0.00,0.0,0.11,3.06,0.57,1.58,2.94,3.51,0.00,0.00,-
1,Brent Burns,CAR,D,26,427.966667,16.460256,0.14,0.84,0.42,0.42,0.98,36.84,5.47,2.56,0.22,14.86,8.97,3.93,0.28,0.56,1.12,0.56,0.28,0.28,0.00,0.0,0.28,4.77,1.54,0.42,2.24,2.94,0.00,0.00,-
2,Corey Perry,EDM,R,26,262.566667,10.098718,0.91,0.00,0.00,0.00,0.91,50.00,4.80,19.05,0.67,11.20,7.77,7.08,3.66,0.00,0.46,3.66,1.14,0.69,0.46,0.0,1.83,1.83,0.91,1.83,4.34,1.14,0.46,0.46,11.43
3,Alex Ovechkin,WSH,L,18,215.250000,11.958333,2.51,1.95,1.11,0.84,4.46,72.73,9.48,26.47,0.91,19.23,13.66,11.15,4.46,0.00,1.39,1.67,0.84,0.84,0.00,0.0,0.00,1.95,1.11,11.43,2.23,0.84,0.00,0.00,-
4,Evgeni Malkin,PIT,C,28,409.850000,14.637500,0.59,1.46,1.02,0.44,2.05,77.78,5.42,10.81,0.77,10.69,7.03,6.29,3.22,0.00,1.32,1.46,0.73,0.73,0.00,0.0,0.88,4.10,1.02,1.90,3.81,2.64,12.59,17.57,6.11


In [22]:
def get_skater_stats(lineup: Lineup, player_stats_df: pd.DataFrame, filter: Optional[str] = None) -> pd.DataFrame:
    """
    Gets stats for players in the lineup, maintaining lineup order.
    
    Args:
        lineup (Lineup): The lineup containing players
        player_stats_df (pd.DataFrame): DataFrame with player statistics
     filter (str, optional): Type of filter to apply on the stats.
            Defaults to None, which keeps all stats. If 'shots', only shot-related statistics are kept.
        
    Returns:
        pd.DataFrame: Player statistics ordered according to lineup positions
    """
    # Apply filter if specified
    if filter == 'shots':
        # Define shot-related columns to keep
        shot_columns = ['player', 'team', 'position', 'gp', 'toi', 'toi/gp', 'icf/60', 'iff/60'] + [col for col in player_stats_df.columns if 'shot' in col.lower()]
        player_stats_df = player_stats_df[shot_columns]
    
    # Create ordered list of players (forwards then defense)
    players = []
    # Add forwards in order
    players.extend([p for p in lineup.forwards if p])
    # Add defense in order 
    players.extend([p for p in lineup.defense if p])
    
    # Create ordered list of player names
    player_names = [player.name for player in players]
    
    # Filter stats and reorder to match lineup order
    stats_df = player_stats_df[player_stats_df['player'].isin(player_names)]
    
    # Create ordering dictionary mapping names to their position in lineup
    name_to_position = {name: idx for idx, name in enumerate(player_names)}
    
    # Sort stats DataFrame based on lineup order and reset index
    return stats_df.assign(
        lineup_order=stats_df['player'].map(name_to_position)
    ).sort_values('lineup_order').drop('lineup_order', axis=1).reset_index(drop=True)

# lineup_player_stats = get_skater_stats(my_lineup, player_stats_df)

In [23]:
def get_goalie_stats(lineup: Lineup, goalie_stats_df: pd.DataFrame) -> pd.DataFrame:
    """
    Gets stats for goalies in the lineup, maintaining lineup order.
    
    Args:
        lineup (Lineup): The lineup containing goalies
        goalie_stats_df (pd.DataFrame): DataFrame with goalie statistics
        
    Returns:
        pd.DataFrame: Goalie statistics ordered according to lineup positions
    """
    # Create ordered list of goalies
    goalies = [goalie for goalie in lineup.goalies if goalie]
    goalie_names = [goalie.name for goalie in goalies]
    
    # Filter stats and reorder to match lineup order
    stats_df = goalie_stats_df[goalie_stats_df['player'].isin(goalie_names)]
    
    # Create ordering dictionary mapping names to their position in lineup
    name_to_position = {name: idx for idx, name in enumerate(goalie_names)}
    
    # Sort stats DataFrame based on lineup order and reset index
    return stats_df.assign(
        lineup_order=stats_df['player'].map(name_to_position)
    ).sort_values('lineup_order').drop('lineup_order', axis=1).reset_index(drop=True)

In [24]:
lineup_goalie_stats = get_goalie_stats(my_lineup, goalie_stats_df)
lineup_goalie_stats

,player,team,gp,toi,toi/gp,shots_against/60,saves/60,sv%,gaa,gsaa/60,xg_against/60,hd_shots_against/60,hd_saves/60,hdsv%,hdgaa,hdgsaa/60,md_shots_against/60,md_saves/60,mdsv%,mdgaa,mdgsaa/60,ld_shots_against/60,ld_saves/60,ldsv%,ldgaa,ldgsaa/60,rush_attempts_against/60,rebound_attempts_against/60,avg._shot_distance,avg._goal_distance
0,Anthony Stolarz,TOR,15,689.633333,45.975556,28.80,27.41,0.952,1.39,1.15,2.52,6.26,5.31,0.847,0.96,0.16,7.05,6.61,0.938,0.44,0.40,13.66,13.66,1.000,0.00,0.44,3.39,5.83,38.77,15.88
1,Joseph Woll,TOR,9,441.733333,49.081481,25.67,23.91,0.931,1.77,0.50,2.02,5.03,4.35,0.865,0.68,0.22,6.52,6.25,0.958,0.27,0.51,12.77,11.95,0.936,0.81,-0.40,2.58,4.75,39.10,27.92


In [25]:
yesterday

'2024-12-06'

In [26]:
get_most_recent_game_id('TOR', '2024-11-21')

2024020301

In [27]:
def extract_team_lineup(team: str, reference_date: Optional[str] = None) -> Lineup:
    """
    Extracts the most recent lineup for the specified team based on the latest game data.

    This function performs the following steps:
        1. Determines the reference date (defaults to yesterday if not provided).
        2. Retrieves the most recent game ID for the team using `get_most_recent_game_id`.
        3. Fetches the game boxscore data using `get_game_boxscore`.
        4. Processes the boxscore to obtain skaters and goalies using `display_boxscore`.
        5. Constructs and returns a `Lineup` object populated with the team's players.

    Args:
        team (str): The three-letter team code (e.g., 'TOR').
        reference_date (Optional[str]): The reference date in 'YYYY-MM-DD' format. Defaults to yesterday's date.

    Returns:
        Lineup: A `Lineup` object containing the team's players from the most recent game.

    Raises:
        ValueError: If no recent game is found for the team or if the team is not part of the retrieved game.
    """
    # Step 1: Determine the reference date
    if reference_date is None:
        today_datetime = datetime.now()
        yesterday_datetime = today_datetime - timedelta(days=1, hours=6)  # Adjust for UTC offset if necessary
        reference_date = yesterday_datetime.strftime('%Y-%m-%d')

    # Step 2: Retrieve the most recent game ID for the team
    game_id = get_most_recent_game_id(team, reference_date)
    if game_id is None:
        raise ValueError(f"No recent game found for team '{team}' before {reference_date}.")

    # Print the game_id
    print(f"Game ID: {game_id}")

    # Step 3: Fetch the game boxscore data
    game_data = get_game_boxscore(game_id, clean=False)

    # Step 4: Process the boxscore to obtain skaters and goalies
    away_skaters, away_goalies, home_skaters, home_goalies = display_boxscore(game_data)

    # Extract team abbrevs to determine if the team is home or away
    away_team_code = game_data.get('awayTeam', {}).get('abbrev')
    home_team_code = game_data.get('homeTeam', {}).get('abbrev')

    if not away_team_code or not home_team_code:
        raise ValueError("Team abbreviations not found in game data.")

    if team.upper() == away_team_code.upper():
        team_side = 'Away'
        skaters = away_skaters
        goalies = away_goalies
    elif team.upper() == home_team_code.upper():
        team_side = 'Home'
        skaters = home_skaters
        goalies = home_goalies
    else:
        raise ValueError(f"Team '{team}' not found in game ID {game_id}.")

    # Step 5: Construct the Lineup object
    lineup = Lineup(name=f"{team.upper()} Lineup from Game {game_id}")

    # Add Skaters to the Lineup
    for _, skater in skaters.iterrows():
        try:
            position_enum = Position(skater['position'])  # Convert to Position Enum
        except ValueError:
            print(f"Invalid position '{skater['position']}' for player '{skater['name']}'. Skipping.")
            continue

        player = Player(
            player_id=skater['playerId'],
            name=get_player_full_name(skater['playerId'], db_config, suppress_log=True),
            team=team.upper(),
            position=position_enum
        )

        # Add player to the appropriate category in the lineup
        if player.position.category == 'F':
            try:
                empty_slot = next(i for i, p in enumerate(lineup.forwards) if p is None)
                lineup.add_forward(player, empty_slot)
            except StopIteration:
                print(f"No available forward slot to add player '{player.name}'.")
        elif player.position.category == 'D':
            try:
                empty_slot = next(i for i, p in enumerate(lineup.defense) if p is None)
                lineup.add_defense(player, empty_slot)
            except StopIteration:
                print(f"No available defense slot to add player '{player.name}'.")
        else:
            print(f"Player '{player.name}' has an unrecognized category '{player.position.category}'. Skipping.")

    # Add Goalies to the Lineup
    for _, goalie in goalies.iterrows():
        player = Player(
            player_id=goalie['playerId'],
            name=get_player_full_name(goalie['playerId'], db_config, suppress_log=True),
            team=team.upper(),
            position=Position.G
        )
        try:
            empty_slot = next(i for i, p in enumerate(lineup.goalies) if p is None)
            lineup.set_goalie(player, empty_slot)
        except StopIteration:
            print(f"No available goalie slot to add player '{player.name}'.")

    return lineup

In [28]:
chicago_lineup = extract_team_lineup('CHI', '2024-11-22')

Game ID: 2024020313
Added player 'Ryan Donato' to Forwards slot 1.
Added player 'Craig Smith' to Forwards slot 2.
Added player 'Jason Dickinson' to Forwards slot 3.
Added player 'Nick Foligno' to Forwards slot 4.
Added player 'Joey Anderson' to Forwards slot 5.
Added player 'Tyler Bertuzzi' to Forwards slot 6.
Added player 'Taylor Hall' to Forwards slot 7.
Added player 'Lukas Reichel' to Forwards slot 8.
Added player 'Patrick Maroon' to Forwards slot 9.
Added player 'Teuvo Teravainen' to Forwards slot 10.
Added player 'Ilya Mikheyev' to Forwards slot 11.
Added player 'Connor Bedard' to Forwards slot 12.
Added player 'Connor Murphy' to Defense slot 1.
Added player 'Alec Martinez' to Defense slot 2.
Added player 'Nolan Allan' to Defense slot 3.
Added player 'Wyatt Kaiser' to Defense slot 4.
Added player 'Alex Vlasic' to Defense slot 5.
Added player 'TJ Brodie' to Defense slot 6.
Added player 'Petr Mrazek' to Goalies slot 1.
Added player 'Arvid Soderblom' to Goalies slot 2.


In [29]:
lineup_skater_stats = get_skater_stats(chicago_lineup, player_stats_df)
lineup_skater_stats

,player,team,position,gp,toi,toi/gp,goals/60,total_assists/60,first_assists/60,second_assists/60,total_points/60,ipp,shots/60,sh%,ixg/60,icf/60,iff/60,iscf/60,ihdcf/60,rush_attempts/60,rebounds_created/60,pim/60,total_penalties/60,minor/60,major/60,misconduct/60,penalties_drawn/60,giveaways/60,takeaways/60,hits/60,hits_taken/60,shots_blocked/60,faceoffs_won/60,faceoffs_lost/60,faceoffs_%
0,Ryan Donato,CHI,C,24,300.166667,12.506944,1.60,0.20,0.20,0.00,1.80,90.00,10.39,15.38,1.08,17.99,13.99,8.60,4.00,0.60,1.20,1.20,0.60,0.60,0.00,0.00,0.60,1.20,1.20,8.60,4.40,1.00,10.39,11.99,9.28
1,Craig Smith,CHI,C,24,244.883333,10.203472,1.23,0.98,0.98,0.00,2.21,75.00,10.78,11.36,0.68,16.17,13.23,7.35,2.94,0.74,1.96,3.92,0.98,0.74,0.00,0.25,0.74,3.19,1.72,4.41,2.70,3.43,0.49,1.72,5.44
2,Jason Dickinson,CHI,C,26,323.000000,12.423077,0.93,0.37,0.37,0.00,1.30,63.64,3.72,25.00,0.41,11.15,6.13,5.76,2.23,0.19,0.93,2.23,1.11,1.11,0.00,0.00,0.56,1.86,1.67,6.50,5.02,3.34,25.26,27.31,8.93
3,Nick Foligno,CHI,L,26,326.266667,12.548718,0.74,0.55,0.37,0.18,1.29,58.33,4.78,15.38,0.58,8.46,6.80,6.25,3.68,0.18,1.47,1.66,0.55,0.37,0.18,0.00,1.29,1.84,0.74,13.61,3.49,1.66,16.55,15.82,9.40
4,Joey Anderson,CHI,R,12,150.833333,12.569444,0.00,0.40,0.40,0.00,0.40,33.33,4.38,0.00,0.43,7.96,5.17,3.98,1.99,0.40,0.80,0.00,0.00,0.00,0.00,0.00,0.40,1.19,2.39,3.18,6.36,2.78,0.00,0.00,-
5,Tyler Bertuzzi,CHI,L,26,312.083333,12.003205,0.19,0.58,0.58,0.00,0.77,40.00,3.85,5.00,0.57,9.04,7.31,4.42,2.88,0.38,0.19,1.15,0.58,0.58,0.00,0.00,1.15,1.92,1.35,3.65,1.92,2.11,0.00,0.00,-
6,Taylor Hall,CHI,L,25,333.216667,13.328667,0.72,0.54,0.18,0.36,1.26,70.00,7.74,9.30,0.61,15.13,10.26,7.56,2.52,0.72,0.36,1.44,0.72,0.72,0.00,0.00,0.72,2.88,2.34,2.88,5.22,3.42,2.34,4.32,6.33
7,Lukas Reichel,CHI,L,22,227.800000,10.354545,0.79,1.32,1.05,0.26,2.11,88.89,8.17,9.68,0.84,12.38,10.01,7.64,3.95,0.79,0.79,1.05,0.53,0.53,0.00,0.00,0.26,0.79,1.58,1.84,3.16,3.16,11.06,22.12,8.78
8,Patrick Maroon,CHI,L,23,238.600000,10.373913,0.25,1.76,1.26,0.50,2.01,80.00,4.53,5.56,0.56,8.80,7.80,4.27,2.51,1.01,0.50,4.53,1.51,1.01,0.50,0.00,1.01,2.51,0.50,10.31,4.02,0.50,2.51,2.51,12.57
9,Teuvo Teravainen,CHI,C,26,328.750000,12.644231,0.37,0.37,0.18,0.18,0.73,44.44,6.57,5.56,0.34,10.59,8.03,4.20,0.55,0.73,0.73,0.73,0.37,0.37,0.00,0.00,0.37,4.02,0.91,0.37,0.91,2.01,0.91,0.37,13.04


In [30]:
lineup_goalie_stats = get_goalie_stats(chicago_lineup, goalie_stats_df)
lineup_goalie_stats

,player,team,gp,toi,toi/gp,shots_against/60,saves/60,sv%,gaa,gsaa/60,xg_against/60,hd_shots_against/60,hd_saves/60,hdsv%,hdgaa,hdgsaa/60,md_shots_against/60,md_saves/60,mdsv%,mdgaa,mdgsaa/60,ld_shots_against/60,ld_saves/60,ldsv%,ldgaa,ldgsaa/60,rush_attempts_against/60,rebound_attempts_against/60,avg._shot_distance,avg._goal_distance
0,Petr Mrazek,CHI,19,924.666667,48.666667,28.88,26.34,0.912,2.53,0.01,2.63,8.05,6.55,0.815,1.49,-0.05,6.29,5.65,0.897,0.65,0.10,12.72,12.33,0.969,0.39,0.02,2.47,7.07,35.91,19.03
1,Arvid Soderblom,CHI,7,350.483333,50.069048,29.10,26.88,0.924,2.23,0.34,2.49,7.36,6.16,0.837,1.20,0.12,7.36,6.68,0.907,0.68,0.19,12.33,11.98,0.972,0.34,0.06,2.74,5.82,37.35,21.92


In [31]:
def calculate_min_percentage(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculates the average time on ice per game as a percentage of total game time (60 minutes).
    
    Args:
        df (pd.DataFrame): DataFrame containing 'toi' and 'gp' columns
        
    Returns:
        pd.DataFrame: Original DataFrame with new 'min%' column added
    """
    df_copy = df.copy()
    df_copy['min%'] = (df_copy['toi'] / df_copy['gp'] / 300 * 100).round(2)
    return df_copy

# # Apply the function to lineup_player_stats
# lineup_player_stats = calculate_min_percentage(lineup_player_stats)
# lineup_player_stats

In [32]:
def sum_min_percentage(df: pd.DataFrame) -> float:
    """
    Calculates the sum of the 'min%' column in the given DataFrame.
    
    Args:
        df (pd.DataFrame): DataFrame containing the 'min%' column.
        
    Returns:
        float: The total sum of the 'min%' values.
        
    Raises:
        KeyError: If the 'min%' column is not present in the DataFrame.
    """
    if 'min%' not in df.columns:
        raise KeyError("The DataFrame does not contain a 'min%' column.")
    
    total_min_percentage = df['min%'].sum()
    return total_min_percentage

# Example usage:
# total_min_percentage = sum_min_percentage(lineup_player_stats)
# print(f"Total min%: {total_min_percentage}")

In [33]:
def calculate_adj_min(df: pd.DataFrame, total_min_percentage: float) -> None:
    """
    Calculates the adjusted minimum (adj_min) for each player based on their min% and the total min%.
    
    The formula used is:
        adj_min = (min% / total_min_percentage) * 300
    
    Args:
        df (pd.DataFrame): DataFrame containing the 'min%' column.
        total_min_percentage (float): The total sum of the 'min%' column.
        
    Raises:
        KeyError: If the 'min%' column is not present in the DataFrame.
        ValueError: If total_min_percentage is not a positive number.
    """
    if 'min%' not in df.columns:
        raise KeyError("The DataFrame does not contain a 'min%' column.")
    
    if total_min_percentage <= 0:
        raise ValueError("total_min_percentage must be a positive number.")
    
    # Calculate and append the 'adj_min' column
    df['adj_min'] = ((df['min%'] / total_min_percentage) * 300).round(2)

# # Example usage:
# calculate_adj_min(lineup_player_stats, total_min_percentage)
# print(lineup_player_stats[['player', 'min%', 'adj_min']])

In [34]:
player_stats_df.head()

,player,team,position,gp,toi,toi/gp,goals/60,total_assists/60,first_assists/60,second_assists/60,total_points/60,ipp,shots/60,sh%,ixg/60,icf/60,iff/60,iscf/60,ihdcf/60,rush_attempts/60,rebounds_created/60,pim/60,total_penalties/60,minor/60,major/60,misconduct/60,penalties_drawn/60,giveaways/60,takeaways/60,hits/60,hits_taken/60,shots_blocked/60,faceoffs_won/60,faceoffs_lost/60,faceoffs_%
0,Ryan Suter,STL,D,27,530.150000,19.635185,0.11,0.45,0.11,0.34,0.57,20.83,2.60,4.35,0.11,7.02,3.96,1.92,0.23,0.34,0.57,1.58,0.79,0.79,0.00,0.0,0.11,3.06,0.57,1.58,2.94,3.51,0.00,0.00,-
1,Brent Burns,CAR,D,26,427.966667,16.460256,0.14,0.84,0.42,0.42,0.98,36.84,5.47,2.56,0.22,14.86,8.97,3.93,0.28,0.56,1.12,0.56,0.28,0.28,0.00,0.0,0.28,4.77,1.54,0.42,2.24,2.94,0.00,0.00,-
2,Corey Perry,EDM,R,26,262.566667,10.098718,0.91,0.00,0.00,0.00,0.91,50.00,4.80,19.05,0.67,11.20,7.77,7.08,3.66,0.00,0.46,3.66,1.14,0.69,0.46,0.0,1.83,1.83,0.91,1.83,4.34,1.14,0.46,0.46,11.43
3,Alex Ovechkin,WSH,L,18,215.250000,11.958333,2.51,1.95,1.11,0.84,4.46,72.73,9.48,26.47,0.91,19.23,13.66,11.15,4.46,0.00,1.39,1.67,0.84,0.84,0.00,0.0,0.00,1.95,1.11,11.43,2.23,0.84,0.00,0.00,-
4,Evgeni Malkin,PIT,C,28,409.850000,14.637500,0.59,1.46,1.02,0.44,2.05,77.78,5.42,10.81,0.77,10.69,7.03,6.29,3.22,0.00,1.32,1.46,0.73,0.73,0.00,0.0,0.88,4.10,1.02,1.90,3.81,2.64,12.59,17.57,6.11


In [35]:
def calculate_ixg_per_60(df: pd.DataFrame) -> None:
    """
    Calculates the expected goals per 60 minutes (ixg_per_60) for each player.

    The formula used is:
        ixg_per_60 = (ixg / toi) * 60

    Args:
        df (pd.DataFrame): DataFrame containing the 'ixg' and 'toi' columns.
        
    Raises:
        KeyError: If either 'ixg' or 'toi' columns are not present in the DataFrame.
        ValueError: If 'toi' contains zero or negative values.
    """
    # Check if required columns exist
    required_columns = {'ixg', 'toi'}
    missing_columns = required_columns - set(df.columns)
    if missing_columns:
        raise KeyError(f"The DataFrame is missing the following required columns: {', '.join(missing_columns)}")
    
    # Check for non-positive 'toi' values to avoid division by zero or negative scaling
    if (df['toi'] <= 0).any():
        raise ValueError("The 'toi' column contains zero or negative values, which are invalid for calculating ixg_per_60.")
    
    # Calculate ixg_per_60 and append as a new column
    df['ixg_per_60'] = (df['ixg'] / df['toi']) * 60
    
    # Optionally, round the values for better readability
    df['ixg_per_60'] = df['ixg_per_60'].round(2)

# Example usage:
# calculate_ixg_per_60(lineup_player_stats)
# print(lineup_player_stats[['player', 'ixg', 'toi', 'ixg_per_60']])

In [36]:
# Define file paths to load the model and transformer
model_filepath = 'models/polynomial_model_degree_1.pkl'
poly_filepath = 'models/polynomial_features_degree_1.pkl'

# Load the regression model
with open(model_filepath, 'rb') as model_file:
    loaded_model = pickle.load(model_file)
print(f"Model loaded from {model_filepath}")

# Load the PolynomialFeatures transformer
with open(poly_filepath, 'rb') as poly_file:
    loaded_poly = pickle.load(poly_file)
print(f"PolynomialFeatures transformer loaded from {poly_filepath}")

# Function to make predictions using the loaded model and transformer
def predict_gpm(new_ixg60_value, model, poly, x_col='ixg60'):
    """
    Predicts 'gpm' using the loaded model and polynomial transformer.
    
    Parameters:
        new_ixg60_value (float): The new ixg60 value for prediction.
        model (RegressionResults): The loaded regression model.
        poly (PolynomialFeatures): The loaded polynomial features transformer.
        x_col (str): The name of the independent variable column. Defaults to 'ixg60'.
        
    Returns:
        predicted_gpm (float): The predicted gpm value.
    """
    # Prepare the input data
    X_new = np.array([[new_ixg60_value]])
    X_new_poly = poly.transform(X_new)
    X_new_poly_const = sm.add_constant(X_new_poly, has_constant='add')
    
    # Create DataFrame with appropriate column names
    feature_names = ['const'] + poly.get_feature_names_out([x_col]).tolist()
    new_data = pd.DataFrame(X_new_poly_const, columns=feature_names)
    
    # Predict
    predicted_gpm = model.predict(new_data)
    return predicted_gpm.iloc[0]

# Example: Predicting 'gpm' for a new ixg60 value
# new_ixg60_value = 50
# predicted_gpm = predict_gpm(new_ixg60_value, loaded_model, loaded_poly)
# print(f"Predicted GPM for ixg60={new_ixg60_value}: {predicted_gpm:.4f}")

Model loaded from models/polynomial_model_degree_1.pkl
PolynomialFeatures transformer loaded from models/polynomial_features_degree_1.pkl


In [37]:
# Function to predict GPM for each player and add it to the DataFrame
def add_gpm_to_lineup(lineup_df, model, poly):
    """
    Adds a 'gpm' column to the lineup_player_stats DataFrame using the predict_gpm function.
    
    Args:
        lineup_df (pd.DataFrame): DataFrame containing 'ixg/60' column.
        model: Loaded regression model.
        poly: Loaded PolynomialFeatures transformer.
        
    Returns:
        pd.DataFrame: Updated DataFrame with 'gpm' column added.
    """
    # Define a helper function to handle potential missing or invalid values
    def safe_predict(ixg_60):
        if pd.isna(ixg_60):
            return np.nan
        try:
            return predict_gpm(ixg_60, model, poly)
        except Exception as e:
            print(f"Error predicting GPM for ixg_60={ixg_60}: {e}")
            return np.nan
    
    # Apply the predict_gpm function to each 'ixg/60' value
    lineup_df['gpm'] = lineup_df['ixg/60'].apply(safe_predict)
    return lineup_df

# # Apply the function to add 'gpm' to your DataFrame
# lineup_player_stats = add_gpm_to_lineup(lineup_player_stats, loaded_model, loaded_poly)

# # Display the updated DataFrame with 'gpm'
# print(lineup_player_stats[['player', 'ixg/60', 'gpm']])

In [38]:
def calculate_x_goals(lineup_stats_df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculates 'x_goals' by multiplying 'gpm' with 'adj_min' for each player.
    
    Args:
        lineup_stats_df (pd.DataFrame): DataFrame containing 'gpm' and 'adj_min' columns.
        
    Returns:
        pd.DataFrame: DataFrame with the new 'x_goals' column added.
        
    Raises:
        KeyError: If 'gpm' or 'adj_min' columns are not present in the DataFrame.
        TypeError: If 'gpm' or 'adj_min' contain non-numeric data.
    """
    # Check if required columns exist
    required_columns = {'gpm', 'adj_min'}
    missing_columns = required_columns - set(lineup_stats_df.columns)
    if missing_columns:
        raise KeyError(f"The DataFrame is missing the following required columns: {', '.join(missing_columns)}")
    
    # Check if 'gpm' and 'adj_min' are numeric
    if not pd.api.types.is_numeric_dtype(lineup_stats_df['gpm']):
        raise TypeError("'gpm' column must be numeric.")
    if not pd.api.types.is_numeric_dtype(lineup_stats_df['adj_min']):
        raise TypeError("'adj_min' column must be numeric.")
    
    # Calculate 'x_goals'
    lineup_stats_df = lineup_stats_df.copy()
    lineup_stats_df['x_goals'] = lineup_stats_df['gpm'] * lineup_stats_df['adj_min']
    
    return lineup_stats_df

In [39]:
def calculate_xg_against_per_60(goalie_stats_df: pd.DataFrame) -> float:
    """
    Calculate the expected goals against (xg_against) per 60 minutes.

    Args:
        goalie_stats_df (pd.DataFrame): DataFrame containing goalie statistics with 'xg_against' and 'toi' columns.

    Returns:
        float: The league average xg_against per 60 minutes.

    Raises:
        KeyError: If required columns are missing from the DataFrame.
        ValueError: If no valid goalies with non-zero 'toi' are found.
    """
    # Ensure required columns are present
    required_columns = {'xg_against', 'toi'}
    missing_columns = required_columns - set(goalie_stats_df.columns)
    if missing_columns:
        raise KeyError(f"Missing columns in goalie_stats_df: {', '.join(missing_columns)}")

    # Drop rows with missing or zero 'toi' to avoid division errors
    valid_goalies = goalie_stats_df.dropna(subset=['xg_against', 'toi'])
    valid_goalies = valid_goalies[valid_goalies['toi'] > 0]

    if valid_goalies.empty:
        raise ValueError("No valid goalies with non-zero 'toi' found in goalie_stats_df.")

    # Calculate xg against per 60 minutes for each goalie
    valid_goalies['xg_against_per_60'] = (valid_goalies['xg_against'] / valid_goalies['toi']) * 60

    return valid_goalies

In [40]:
def calculate_league_avg_xg_against_per_60(goalie_stats_df: pd.DataFrame) -> float:
    """
    Calculate the league average expected goals against (xg_against) per 60 minutes.

    Args:
        goalie_stats_df (pd.DataFrame): DataFrame containing goalie statistics with 'xg_against' and 'toi' columns.

    Returns:
        float: The league average xg_against per 60 minutes.

    Raises:
        KeyError: If required columns are missing from the DataFrame.
        ValueError: If no valid goalies with non-zero 'toi' are found.
    """
    # Ensure required columns are present
    required_columns = {'xg_against', 'toi'}
    missing_columns = required_columns - set(goalie_stats_df.columns)
    if missing_columns:
        raise KeyError(f"Missing columns in goalie_stats_df: {', '.join(missing_columns)}")

    # Drop rows with missing or zero 'toi' to avoid division errors
    valid_goalies = goalie_stats_df.dropna(subset=['xg_against', 'toi'])
    valid_goalies = valid_goalies[valid_goalies['toi'] > 0]

    if valid_goalies.empty:
        raise ValueError("No valid goalies with non-zero 'toi' found in goalie_stats_df.")

    # Calculate xg against per 60 minutes for each goalie
    valid_goalies['xg_against_per_60'] = (valid_goalies['xg_against'] / valid_goalies['toi']) * 60

    # Calculate the league average
    league_avg_xg_against_per_60 = valid_goalies['xg_against_per_60'].mean()

    return league_avg_xg_against_per_60

In [41]:
def calculate_xg_against_adj_percentage(lineup_goalie_stats, goalie_avg_xg_against_per_60):
    """
    Calculate the expected goals against adjusted percentage.
    """
    lineup_goalie_stats['adj%'] = (lineup_goalie_stats['xg_against/60'] / goalie_avg_xg_against_per_60) * 100
    return lineup_goalie_stats

In [42]:
def process_team_lineup(input_date: str, team: str, db_config: Dict[str, str], model, poly) -> pd.DataFrame:
    """
    Processes the team's lineup for a given date and predicts GPM for each player.
    This function performs the following steps:
        1. Calls `nst_on_ice_scraper` for the input date minus one day.
        2. Extracts the team's lineup using `extract_team_lineup`.
        3. Retrieves skater statistics with `get_skater_stats`.
        4. Calculates `min%` using `calculate_min_percentage`.
        5. Computes `adj_min` using `calculate_adj_min`.
        6. Determines `ixg_per_60` using `calculate_ixg_per_60`.
        7. Predicts `gpm` using the loaded polynomial regression model.
        8. Calculcates x_goals by multiplying gpm by adj_min    
    Args:
        input_date (str): The reference date in 'YYYY-MM-DD' format.
        team (str): The three-letter team code (e.g., 'TOR').
        db_config (Dict[str, str]): Database configuration dictionary.
        model: Loaded regression model.
        poly: Loaded PolynomialFeatures transformer.
        
    Returns:
        pd.DataFrame: Updated DataFrame with calculated metrics and predicted GPM.
    
    Raises:
        ValueError: If any step in the data processing pipeline fails.
    """

    try:
        # Step 1: Calculate the date minus one day
        reference_datetime = datetime.strptime(input_date, '%Y-%m-%d') - timedelta(days=1)
        reference_date_str = reference_datetime.strftime('%Y-%m-%d')
        print(f"Fetching data for reference date: {reference_date_str}")

        # Step 2: Call nst_on_ice_scraper for player and goalie stats
        player_stats_df = nst_on_ice_scraper(
            fromseason=20242025,
            thruseason=20242025,
            startdate='',
            enddate=reference_date_str,
            rate='y',
            stdoi='std'
        )
        goalie_stats_df = nst_on_ice_scraper(
            fromseason=20242025,
            thruseason=20242025,
            startdate='',
            enddate=reference_date_str,
            pos='g',
            rate='y',
            stdoi='g'
        )
        print("Player and goalie statistics fetched successfully.")

        # Step 3: Extract team lineup for the input date and team
        lineup = extract_team_lineup(team, input_date)
        print(f"Lineup extracted for team {team} on {input_date}.")

        # Step 4: Get skater statistics for the lineup
        lineup_skater_stats = get_skater_stats(lineup, player_stats_df)

        # Step 5: Calculate min%
        lineup_skater_stats = calculate_min_percentage(lineup_skater_stats)

        # Step 6: Calculate adj_min
        total_min_percentage = sum_min_percentage(lineup_skater_stats)
        calculate_adj_min(lineup_skater_stats, total_min_percentage)

        # Step 7: Calculate ixg_per_60
        # calculate_ixg_per_60(lineup_skater_stats)

        # Step 8: Predict gpm using the polynomial regression model
        lineup_skater_stats = add_gpm_to_lineup(lineup_skater_stats, model, poly)

        # Step 9: Calculate x_goals
        lineup_skater_stats = calculate_x_goals(lineup_skater_stats)

        # Step 10: Get goalie stats for the lineup
        lineup_goalie_stats = get_goalie_stats(lineup, goalie_stats_df)

        # Step 11: Calculate xg_against_per_60
        # lineup_goalie_stats = calculate_xg_against_per_60(lineup_goalie_stats)
        
        # Step 12: Calculate league average xg_against_per_60
        goalie_avg_xg_against_per_60 = goalie_stats_df['xg_against/60'].mean()

        lineup_goalie_stats = calculate_xg_against_adj_percentage(lineup_goalie_stats, goalie_avg_xg_against_per_60)
        
        return lineup_skater_stats, lineup_goalie_stats

    except Exception as e:
        print(f"An error occurred during processing: {e}")
        raise

In [43]:
def load_models(model_filepath='models/polynomial_model_degree_1.pkl',
               poly_filepath='models/polynomial_features_degree_1.pkl'):
    """
    Loads the regression model and PolynomialFeatures transformer from the specified file paths.

    Args:
        model_filepath (str): Path to the saved regression model pickle file.
        poly_filepath (str): Path to the saved PolynomialFeatures transformer pickle file.

    Returns:
        tuple: A tuple containing the loaded regression model and PolynomialFeatures transformer.

    Raises:
        FileNotFoundError: If either of the specified files does not exist.
        pickle.UnpicklingError: If there is an error unpickling the files.
    """
    try:
        # Load the regression model
        with open(model_filepath, 'rb') as model_file:
            loaded_model = pickle.load(model_file)
        print(f"Model loaded from {model_filepath}")

        # Load the PolynomialFeatures transformer
        with open(poly_filepath, 'rb') as poly_file:
            loaded_poly = pickle.load(poly_file)
        print(f"PolynomialFeatures transformer loaded from {poly_filepath}")

        return loaded_model, loaded_poly

    except FileNotFoundError as fnf_error:
        print(f"Error: {fnf_error}")
        raise
    except pickle.UnpicklingError as pickle_error:
        print(f"Error loading pickle files: {pickle_error}")
        raise
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        raise

In [44]:
# Load the models using the new function
loaded_model, loaded_poly = load_models()

# Call the function with desired date and team
input_date = '2024-12-02'
team = 'TOR'
lineup_skater_stats, lineup_goalie_stats = process_team_lineup(input_date, team, db_config, loaded_model, loaded_poly)


Model loaded from models/polynomial_model_degree_1.pkl
PolynomialFeatures transformer loaded from models/polynomial_features_degree_1.pkl
Fetching data for reference date: 2024-12-01
Player and goalie statistics fetched successfully.
Game ID: 2024020378
Added player 'Mitch Marner' to Forwards slot 1.
Added player 'Steven Lorentz' to Forwards slot 2.
Added player 'Matthew Knies' to Forwards slot 3.
Added player 'Connor Dewar' to Forwards slot 4.
Added player 'Pontus Holmberg' to Forwards slot 5.
Added player 'Auston Matthews' to Forwards slot 6.
Added player 'Fraser Minten' to Forwards slot 7.
Added player 'Nikita Grebenkin' to Forwards slot 8.
Added player 'William Nylander' to Forwards slot 9.
Added player 'Nicholas Robertson' to Forwards slot 10.
Added player 'John Tavares' to Forwards slot 11.
Added player 'Alex Nylander' to Forwards slot 12.
Added player 'Simon Benoit' to Defense slot 1.
Added player 'Chris Tanev' to Defense slot 2.
Added player 'Jake McCabe' to Defense slot 3.
Add

In [45]:
lineup_skater_stats.head()

,player,team,position,gp,toi,toi/gp,goals/60,total_assists/60,first_assists/60,second_assists/60,total_points/60,ipp,shots/60,sh%,ixg/60,icf/60,iff/60,iscf/60,ihdcf/60,rush_attempts/60,rebounds_created/60,pim/60,total_penalties/60,minor/60,major/60,misconduct/60,penalties_drawn/60,giveaways/60,takeaways/60,hits/60,hits_taken/60,shots_blocked/60,faceoffs_won/60,faceoffs_lost/60,faceoffs_%,min%,adj_min,gpm,x_goals
0,Mitch Marner,TOR,R,26,363.666667,13.987179,0.49,1.98,1.32,0.66,2.47,83.33,5.94,8.33,0.54,11.88,7.26,6.60,1.65,0.66,1.48,0.33,0.16,0.16,0.0,0.0,0.66,3.79,2.14,1.98,1.81,1.98,0.16,0.49,4.12,4.66,18.06,0.008805,0.159027
1,Steven Lorentz,TOR,C,26,238.183333,9.160897,0.50,0.50,0.00,0.50,1.01,57.14,4.79,10.53,0.75,11.84,8.56,5.29,2.77,0.25,0.76,0.00,0.00,0.00,0.0,0.0,0.76,1.51,1.51,16.37,5.79,5.04,1.26,1.76,10.50,3.05,11.82,0.012200,0.144209
2,Matthew Knies,TOR,L,24,325.700000,13.570833,1.47,0.37,0.18,0.18,1.84,58.82,6.08,24.24,0.81,10.87,8.47,7.92,5.16,0.74,1.47,1.11,0.55,0.55,0.0,0.0,0.74,2.21,0.74,9.58,4.79,1.47,0.00,0.00,-,4.52,17.52,0.013170,0.230746
3,Connor Dewar,TOR,C,10,94.016667,9.401667,0.00,0.00,0.00,0.00,0.00,-,4.47,0.00,0.78,12.76,7.02,7.02,3.83,0.00,2.55,0.00,0.00,0.00,0.0,0.0,0.64,5.11,0.64,14.04,7.66,3.83,10.21,19.78,21.73,3.13,12.13,0.012685,0.153874
4,Pontus Holmberg,TOR,R,24,273.516667,11.396528,0.00,0.88,0.22,0.66,0.88,66.67,3.29,0.00,0.33,9.21,5.70,4.61,1.97,0.22,0.22,2.63,1.32,1.32,0.0,0.0,1.32,2.41,0.88,1.75,7.90,2.85,9.65,9.43,11.09,3.80,14.73,0.005411,0.079697


In [46]:
lineup_goalie_stats

,player,team,gp,toi,toi/gp,shots_against/60,saves/60,sv%,gaa,gsaa/60,xg_against/60,hd_shots_against/60,hd_saves/60,hdsv%,hdgaa,hdgsaa/60,md_shots_against/60,md_saves/60,mdsv%,mdgaa,mdgsaa/60,ld_shots_against/60,ld_saves/60,ldsv%,ldgaa,ldgsaa/60,rush_attempts_against/60,rebound_attempts_against/60,avg._shot_distance,avg._goal_distance,adj%
0,Anthony Stolarz,TOR,15,689.633333,45.975556,28.80,27.41,0.952,1.39,1.15,2.52,6.26,5.31,0.847,0.96,0.16,7.05,6.61,0.938,0.44,0.40,13.66,13.66,1.000,0.00,0.44,3.39,5.83,38.77,15.88,101.926285
1,Joseph Woll,TOR,9,441.733333,49.081481,25.67,23.91,0.931,1.77,0.50,2.02,5.03,4.35,0.865,0.68,0.22,6.52,6.25,0.958,0.27,0.51,12.77,11.95,0.936,0.81,-0.40,2.58,4.75,39.10,27.92,81.702816


In [47]:
from pbp_utils import get_matchup_games

temp_data = get_matchup_games('2024-12-05', '2024-12-05')
pd.DataFrame(list(zip(temp_data['game_ids']['id'], temp_data['game_ids']['date'])), columns=['game_id', 'game_date'])

,game_id,game_date
0,2024020410,2024-12-05
1,2024020411,2024-12-05
2,2024020412,2024-12-05
3,2024020413,2024-12-05
4,2024020414,2024-12-05
5,2024020415,2024-12-05
6,2024020416,2024-12-05
7,2024020417,2024-12-05
8,2024020418,2024-12-05


In [48]:
temp_data

{'next_start_date': '2024-12-12',
 'game_ids': {'id': [2024020410,
   2024020411,
   2024020412,
   2024020413,
   2024020414,
   2024020415,
   2024020416,
   2024020417,
   2024020418],
  'date': ['2024-12-05',
   '2024-12-05',
   '2024-12-05',
   '2024-12-05',
   '2024-12-05',
   '2024-12-05',
   '2024-12-05',
   '2024-12-05',
   '2024-12-05']}}

In [49]:
from game_utils import get_game_boxscore

get_game_boxscore(2024020417, clean=True)

{'away_team': 'STL', 'home_team': 'CGY'}

In [50]:
def process_matchups_for_date(
    input_date: str,
    db_config: Dict[str, str],
    model,
    poly
) -> pd.DataFrame:
    """
    Processes all matchup games for a given date by extracting team lineups and their statistics.
    
    Args:
        input_date (str): The reference date in 'YYYY-MM-DD' format.
        db_config (Dict[str, str]): Database configuration dictionary.
        model: Loaded regression model.
        poly: Loaded PolynomialFeatures transformer.
    
    Returns:
        pd.DataFrame: A DataFrame with columns 'date', 'game_id', 'away_team', 'home_team',
                      'away_x_goals', 'away_adj%', 'home_x_goals', 'home_adj%'.
    """
    try:
        # Step 1: Call get_matchup_games with the input date as both arguments
        temp_data = get_matchup_games(input_date, input_date)
        game_ids = temp_data.get('game_ids', {}).get('id', [])
        game_dates = temp_data.get('game_ids', {}).get('date', [])
    
        if not game_ids:
            print(f"No games found for the date {input_date}.")
            return pd.DataFrame(columns=['date', 'game_id', 'away_team', 'home_team',
                                         'away_x_goals', 'away_adj%', 'home_x_goals', 'home_adj%'])
    
        if len(game_ids) != len(game_dates):
            print("Mismatch between number of game IDs and game dates.")
            return pd.DataFrame(columns=['date', 'game_id', 'away_team', 'home_team',
                                         'away_x_goals', 'away_adj%', 'home_x_goals', 'home_adj%'])
    
        results = []
    
        # Step 2: Iterate through each game_id and corresponding game_date
        for game_id, game_date in zip(game_ids, game_dates):
            print(f"\nProcessing Game ID: {game_id} on Date: {game_date}")
    
            # Step 3: Get the cleaned boxscore for the current game
            boxscore = get_game_boxscore(game_id, clean=True)
    
            # Extract away and home team abbreviations
            away_team = boxscore.get('away_team')
            home_team = boxscore.get('home_team')
    
            if not away_team or not home_team:
                print(f"Could not extract teams for Game ID: {game_id}. Skipping.")
                continue
    
            print(f"Away Team: {away_team}, Home Team: {home_team}")
    
            # Step 4: Process lineup for the away team
            print(f"Processing lineup for Away Team: {away_team}")
            away_skater_stats, away_goalie_stats = process_team_lineup(
                game_date,
                away_team,
                db_config,
                model,
                poly
            )
    
            # Step 5: Process lineup for the home team
            print(f"Processing lineup for Home Team: {home_team}")
            home_skater_stats, home_goalie_stats = process_team_lineup(
                game_date,
                home_team,
                db_config,
                model,
                poly
            )
    
            # Step 6: Sum x_goals and extract adj% for away team
            away_x_goals = away_skater_stats['x_goals'].sum()
            away_adj_percentage = away_goalie_stats.iloc[0]['adj%'] if not away_goalie_stats.empty else np.nan
    
            # Step 7: Sum x_goals and extract adj% for home team
            home_x_goals = home_skater_stats['x_goals'].sum()
            home_adj_percentage = home_goalie_stats.iloc[0]['adj%'] if not home_goalie_stats.empty else np.nan
    
            # Step 8: Append the results with the new columns
            results.append({
                'date': game_date,
                'game_id': game_id,
                'away_team': away_team,
                'home_team': home_team,
                'away_x_goals': away_x_goals,
                'away_adj%': away_adj_percentage,
                'home_x_goals': home_x_goals,
                'home_adj%': home_adj_percentage
            })
    
        # Convert results to DataFrame with the specified column order
        results_df = pd.DataFrame(results, columns=[
            'date', 'game_id', 'away_team', 'home_team',
            'away_x_goals', 'away_adj%', 'home_x_goals', 'home_adj%'
        ])
    
        print("\nAll matchups processed successfully.")
        return results_df
    
    except Exception as e:
        print(f"An error occurred while processing matchups for date {input_date}: {e}")
        return pd.DataFrame(columns=[
            'date', 'game_id', 'away_team', 'home_team',
            'away_x_goals', 'away_adj%', 'home_x_goals', 'home_adj%'
        ])

In [51]:
matchups = process_matchups_for_date('2024-12-04', db_config, loaded_model, loaded_poly)


Processing Game ID: 2024020406 on Date: 2024-12-04
Away Team: NSH, Home Team: TOR
Processing lineup for Away Team: NSH
Fetching data for reference date: 2024-12-03
Player and goalie statistics fetched successfully.
Game ID: 2024020384
Added player 'Filip Forsberg' to Forwards slot 1.
Added player 'Colton Sissons' to Forwards slot 2.
Added player 'Gustav Nyquist' to Forwards slot 3.
Added player 'Mark Jankowski' to Forwards slot 4.
Added player 'Cole Smith' to Forwards slot 5.
Added player 'Fedor Svechkov' to Forwards slot 6.
Added player 'Zachary L'Heureux' to Forwards slot 7.
Added player 'Luke Evangelista' to Forwards slot 8.
Added player 'Jonathan Marchessault' to Forwards slot 9.
Added player 'Tommy Novak' to Forwards slot 10.
Added player 'Ryan O'Reilly' to Forwards slot 11.
Added player 'Steven Stamkos' to Forwards slot 12.
Added player 'Luke Schenn' to Defense slot 1.
Added player 'Nick Blankenburg' to Defense slot 2.
Added player 'Alexandre Carrier' to Defense slot 3.
Added pl

In [52]:
def calculate_predicted_goals(matchups_df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate predicted goals by multiplying x_goals with opposing team's goalie adjustment percentage.
    
    Args:
        matchups_df: DataFrame containing x_goals and adj% columns from process_matchups_for_date
        
    Returns:
        DataFrame with added predicted goals columns
    """
    # Convert percentages to multipliers (e.g. 105% -> 1.05)
    matchups_df = matchups_df.copy()
    matchups_df['away_adj%'] = matchups_df['away_adj%'] / 100
    matchups_df['home_adj%'] = matchups_df['home_adj%'] / 100
    
    # Calculate predicted goals
    matchups_df['away_pred_goals'] = matchups_df['away_x_goals'] * matchups_df['home_adj%']
    matchups_df['home_pred_goals'] = matchups_df['home_x_goals'] * matchups_df['away_adj%']
    
    return matchups_df


In [53]:
matchups = calculate_predicted_goals(matchups)
matchups

,date,game_id,away_team,home_team,away_x_goals,away_adj%,home_x_goals,home_adj%,away_pred_goals,home_pred_goals
0,2024-12-04,2024020406,NSH,TOR,2.288091,0.914101,2.356907,1.019263,2.332166,2.154451
1,2024-12-04,2024020407,BOS,CHI,2.394648,0.849386,2.090766,1.063754,2.547317,1.775867
2,2024-12-04,2024020408,VGK,ANA,2.405855,1.088023,2.390664,1.375196,3.308522,2.601097
3,2024-12-04,2024020409,DAL,LAK,2.486017,0.954548,2.574731,1.075889,2.674678,2.457703


In [54]:
def calculate_win_probabilities(matchups_df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate home and away win probabilities based on predicted goals.
    
    Args:
        matchups_df (pd.DataFrame): DataFrame containing 'home_pred_goals' and 'away_pred_goals'.
    
    Returns:
        pd.DataFrame: DataFrame with added 'home_win_prob' and 'away_win_prob' columns, and selected columns.
    """
    HIA = 0.28
    HOME = matchups_df['home_pred_goals']
    AWAY = matchups_df['away_pred_goals']
    
    matchups_df['home_win_prob'] = 1 / (1 + np.exp((-(HIA + HOME - AWAY) / 1.318)))
    matchups_df['away_win_prob'] = 1 - matchups_df['home_win_prob']
    
    return matchups_df[[
        'date', 'game_id', 'away_team', 'home_team', 
        'away_x_goals', 'away_adj%', 'home_x_goals', 'home_adj%', 
        'away_pred_goals', 'home_pred_goals',
        'away_win_prob', 'home_win_prob'
    ]]

In [55]:
matchups = calculate_win_probabilities(matchups)
matchups

,date,game_id,away_team,home_team,away_x_goals,away_adj%,home_x_goals,home_adj%,away_pred_goals,home_pred_goals,away_win_prob,home_win_prob
0,2024-12-04,2024020406,NSH,TOR,2.288091,0.914101,2.356907,1.019263,2.332166,2.154451,0.480608,0.519392
1,2024-12-04,2024020407,BOS,CHI,2.394648,0.849386,2.090766,1.063754,2.547317,1.775867,0.592154,0.407846
2,2024-12-04,2024020408,VGK,ANA,2.405855,1.088023,2.390664,1.375196,3.308522,2.601097,0.580371,0.419629
3,2024-12-04,2024020409,DAL,LAK,2.486017,0.954548,2.574731,1.075889,2.674678,2.457703,0.488047,0.511953


In [56]:
def calculate_x_shots(lineup_stats_df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculates 'x_shots' by multiplying 'shots/60' divided by 60 with 'adj_min' for each player.
    
    Args:
        lineup_stats_df (pd.DataFrame): DataFrame containing 'shots/60' and 'adj_min' columns.
        
    Returns:
        pd.DataFrame: DataFrame with the new 'x_goals' column added.
        
    Raises:
        KeyError: If 'shots/60' or 'adj_min' columns are not present in the DataFrame.
        TypeError: If 'shots/60' or 'adj_min' contain non-numeric data.
    """
    # Check if required columns exist
    required_columns = {'shots/60', 'adj_min'}
    missing_columns = required_columns - set(lineup_stats_df.columns)
    if missing_columns:
        raise KeyError(f"The DataFrame is missing the following required columns: {', '.join(missing_columns)}")
    
    # Check if 'shots/60' and 'adj_min' are numeric
    if not pd.api.types.is_numeric_dtype(lineup_stats_df['shots/60']):
        raise TypeError("'shots/60' column must be numeric.")
    if not pd.api.types.is_numeric_dtype(lineup_stats_df['adj_min']):
        raise TypeError("'adj_min' column must be numeric.")
    
    # Calculate 'x_shots'
    lineup_stats_df = lineup_stats_df.copy()
    lineup_stats_df['x_shots'] = (lineup_stats_df['shots/60'] / 60) * lineup_stats_df['adj_min']
    
    return lineup_stats_df

In [57]:
def process_team_lineup_shots(input_date: str, team: str, db_config: Dict[str, str]) -> pd.DataFrame:
    """
    Processes the team's lineup for a given date and predicts shots for each player.
    This function performs the following steps:
        Calls `nst_on_ice_scraper` for the input date minus one day. Extracts the team's lineup using `extract_team_lineup`.
        Retrieves skater statistics with `get_skater_stats`. Calculates `min%` using `calculate_min_percentage`.
        Computes `adj_min` using `calculate_adj_min`. Calculcates x_shots by multiplying shots/60 by adj_min    
    Args:
        input_date (str): The reference date in 'YYYY-MM-DD' format.
        team (str): The three-letter team code (e.g., 'TOR').
        db_config (Dict[str, str]): Database configuration dictionary.
    Returns:
        pd.DataFrame: Updated DataFrame with calculated metrics and predicted shots (x_shots).
    
    Raises:
        ValueError: If any step in the data processing pipeline fails.
    """

    try:
        # Step 1: Calculate the date minus one day
        reference_datetime = datetime.strptime(input_date, '%Y-%m-%d') - timedelta(days=1)
        reference_date_str = reference_datetime.strftime('%Y-%m-%d')
        print(f"Fetching data for reference date: {reference_date_str}")

        # Step 2: Call nst_on_ice_scraper for player and goalie stats
        player_stats_df = nst_on_ice_scraper(
            fromseason=20242025,
            thruseason=20242025,
            startdate='',
            enddate=reference_date_str,
            rate='y'
        )
        print("Player and goalie statistics fetched successfully.")

        # Step 3: Extract team lineup for the input date and team
        lineup = extract_team_lineup(team, input_date)
        print(f"Lineup extracted for team {team} on {input_date}.")

        # Step 4: Get skater statistics for the lineup
        lineup_skater_stats = get_skater_stats(lineup, player_stats_df, filter='shots')

        # Step 5: Calculate min%
        lineup_skater_stats = calculate_min_percentage(lineup_skater_stats)

        # Step 6: Calculate adj_min
        total_min_percentage = sum_min_percentage(lineup_skater_stats)
        calculate_adj_min(lineup_skater_stats, total_min_percentage)

        # Step 7: Calculate x_shots
        lineup_skater_stats = calculate_x_shots(lineup_skater_stats)
        
        return lineup_skater_stats

    except Exception as e:
        print(f"An error occurred during processing: {e}")
        raise

In [58]:
shots = process_team_lineup_shots('2024-12-05', 'FLA', db_config)
shots

Fetching data for reference date: 2024-12-04
Player and goalie statistics fetched successfully.
Game ID: 2024020399
Added player 'Sam Bennett' to Forwards slot 1.
Added player 'A.J. Greer' to Forwards slot 2.
Added player 'Jonah Gadjovich' to Forwards slot 3.
Added player 'Sam Reinhart' to Forwards slot 4.
Added player 'Anton Lundell' to Forwards slot 5.
Added player 'Aleksander Barkov' to Forwards slot 6.
Added player 'Evan Rodrigues' to Forwards slot 7.
Added player 'Matthew Tkachuk' to Forwards slot 8.
Added player 'Carter Verhaeghe' to Forwards slot 9.
Added player 'Eetu Luostarinen' to Forwards slot 10.
Added player 'Jesper Boqvist' to Forwards slot 11.
Added player 'Tomas Nosek' to Forwards slot 12.
Added player 'Aaron Ekblad' to Defense slot 1.
Added player 'Dmitry Kulikov' to Defense slot 2.
Added player 'Adam Boqvist' to Defense slot 3.
Added player 'Gustav Forsling' to Defense slot 4.
Added player 'Niko Mikkola' to Defense slot 5.
Added player 'Nate Schmidt' to Defense slot 6

,player,team,position,gp,toi,toi/gp,icf/60,iff/60,shots/60,shots_blocked/60,min%,adj_min,x_shots
0,Sam Bennett,FLA,C,26,364.100000,14.003846,15.66,12.36,9.56,2.80,4.67,17.49,2.786740
1,A.J. Greer,FLA,L,27,260.333333,9.641975,12.45,10.14,7.84,2.07,3.21,12.02,1.570613
2,Jonah Gadjovich,FLA,L,14,100.366667,7.169048,11.36,7.17,2.99,1.20,2.39,8.95,0.446008
3,Sam Reinhart,FLA,C,27,380.100000,14.077778,13.89,11.21,7.58,2.37,4.69,17.56,2.218413
4,Anton Lundell,FLA,C,26,342.666667,13.179487,13.13,10.16,7.35,3.15,4.39,16.44,2.013900
5,Aleksander Barkov,FLA,C,19,253.800000,13.357895,14.18,11.82,6.86,3.55,4.45,16.66,1.904793
6,Evan Rodrigues,FLA,C,27,377.450000,13.979630,13.35,9.38,6.20,0.95,4.66,17.45,1.803167
7,Matthew Tkachuk,FLA,L,22,291.383333,13.244697,13.59,10.91,7.62,1.44,4.41,16.51,2.096770
8,Carter Verhaeghe,FLA,C,27,394.216667,14.600617,18.26,13.09,9.89,1.67,4.87,18.24,3.006560
9,Eetu Luostarinen,FLA,C,27,346.000000,12.814815,8.50,6.42,4.16,2.43,4.27,15.99,1.108640


In [59]:
from nst_scraper import nst_team_on_ice_scraper

team_df = nst_team_on_ice_scraper(
    fromseason=20242025,
    thruseason=20242025,
    startdate='',
    enddate=''
)
team_df

,team,gp,toi,w,l,otl,row,points,point_%,cf,ca,cf%,ff,fa,ff%,sf,sa,sf%,gf,ga,gf%,xgf,xga,xgf%,scf,sca,scf%,scsf,scsa,scsf%,scgf,scga,scgf%,scsh%,scsv%,hdcf,hdca,hdcf%,hdsf,hdsa,hdsf%,hdgf,hdga,hdgf%,hdsh%,hdsv%,mdcf,mdca,mdcf%,mdsf,mdsa,mdsf%,mdgf,mdga,mdgf%,mdsh%,mdsv%,ldcf,ldca,ldcf%,ldsf,ldsa,ldsf%,ldgf,ldga,ldgf%,ldsh%,ldsv%,sh%,sv%,pdo
0,Anaheim Ducks,25,1516:48,10,12,3,9,23,0.460,1487,1771,45.64,1080,1274,45.88,688,834,45.20,61,76,44.53,75.88,95.93,44.16,661,851,43.72,350,435,44.59,48,65,42.48,13.71,85.06,272,374,42.11,178,232,43.41,33,42,44.00,18.54,81.90,389,477,44.92,172,203,45.87,15,23,39.47,8.72,88.67,864,1085,44.33,291,356,44.98,10,8,55.56,3.44,97.75,8.87,90.89,0.998
1,Boston Bruins,28,1698:28,14,11,3,14,31,0.554,1571,1577,49.90,1126,1134,49.82,778,781,49.90,71,84,45.81,85.28,72.79,53.95,784,727,51.89,415,382,52.07,59,71,45.38,14.22,81.41,315,274,53.48,212,175,54.78,34,41,45.33,16.04,76.57,469,453,50.87,203,207,49.51,25,30,45.45,12.32,85.51,914,962,48.72,313,346,47.50,10,9,52.63,3.19,97.40,9.13,89.24,0.984
2,Buffalo Sabres,26,1580:09,11,12,3,10,25,0.481,1625,1502,51.97,1146,1096,51.12,743,753,49.67,77,81,48.73,77.17,79.37,49.30,727,724,50.10,359,404,47.05,62,72,46.27,17.27,82.18,285,292,49.39,164,185,46.99,38,45,45.78,23.17,75.68,442,432,50.57,195,219,47.10,24,27,47.06,12.31,87.67,1003,914,52.32,338,302,52.81,12,8,60.00,3.55,97.35,10.36,89.24,0.996
3,Carolina Hurricanes,26,1570:40,17,8,1,17,35,0.673,1805,1262,58.85,1315,951,58.03,844,656,56.27,98,76,56.32,94.00,76.51,55.13,866,637,57.62,441,377,53.91,71,54,56.80,16.10,85.68,344,303,53.17,220,217,50.34,44,32,57.89,20.00,85.25,522,334,60.98,221,160,58.01,27,22,55.10,12.22,86.25,1118,734,60.37,375,241,60.88,24,20,54.55,6.40,91.70,11.61,88.41,1.000
4,Columbus Blue Jackets,26,1572:05,11,12,3,10,25,0.481,1610,1542,51.08,1157,1104,51.17,806,741,52.10,88,95,48.09,77.96,78.72,49.76,711,721,49.65,404,375,51.86,69,73,48.59,17.08,80.53,286,325,46.81,195,213,47.79,33,46,41.77,16.92,78.40,425,396,51.77,209,162,56.33,36,27,57.14,17.22,83.33,980,931,51.28,346,337,50.66,17,20,45.95,4.91,94.07,10.92,87.18,0.981
5,Calgary Flames,27,1653:08,13,9,5,10,31,0.574,1735,1647,51.30,1193,1160,50.70,805,806,49.97,69,77,47.26,80.25,78.74,50.47,790,791,49.97,408,420,49.28,50,60,45.45,12.25,85.71,312,291,51.74,195,192,50.39,26,35,42.62,13.33,81.77,478,500,48.88,213,228,48.30,24,25,48.98,11.27,89.04,1048,1030,50.43,348,335,50.95,16,16,50.00,4.60,95.22,8.57,90.45,0.990
6,Chicago Blackhawks,26,1568:34,8,16,2,7,18,0.346,1345,1649,44.92,963,1154,45.49,675,792,46.01,63,82,43.45,66.14,85.79,43.53,608,778,43.87,334,409,44.95,48,64,42.86,14.37,84.35,245,321,43.29,157,222,41.42,27,45,37.50,17.20,79.73,363,457,44.27,177,187,48.63,21,19,52.50,11.86,89.84,839,1016,45.23,311,331,48.44,14,14,50.00,4.50,95.77,9.33,89.65,0.990
7,Colorado Avalanche,27,1632:06,14,13,0,13,28,0.519,1711,1512,53.09,1184,1060,52.76,802,711,53.01,88,102,46.32,87.79,82.16,51.66,871,692,55.73,460,361,56.03,64,74,46.38,13.91,79.50,339,309,52.31,234,194,54.67,36,52,40.91,15.38,73.20,532,383,58.14,226,167,57.51,28,22,56.00,12.39,86.83,948,928,50.53,304,304,50.00,21,22,48.84,6.91,92.76,10.97,85.65,0.966
8,Dallas Stars,26,1565:00,16,10,0,15,32,0.615,1639,1420,53.58,1142,1046,52.19,785,708,52.58,85,67,55.92,87.38,75.84,53.54,802,691,53.72,421,358,54.04,69,54,56.10,16.39,84.92,335,290,53.60,225,180,55.56,42,30,58.33,18.67,83.33,467,401,53.80,196,178,52.41,27,24,52.94,13.78,86.52,968,872,52.61,319,307,50.96,13,12,52.00,4.08,96.09,10.83,90.54,1.014
9,Detroit Red Wings,26,1572:24,10,12,4,10,24,0.462,1418,1681,45.76,997,1219,44.99,649,819,44.21,66,81,44.90,67.07,81.88,45.03,661,772,46.13,355,392,47.52,55,56,49.55,15.49,85.71,237,284,45.49,145,174,45.45,22,30,42.31,15.17,82.76,424,488,46.49,210,218,49.07,33,26,55.93,15.71,88.07,867,1028,45.75,253,375,40.29,8,23,25.81,3.16,93.87,10.17,90.11,1.003


In [60]:
from team_utils import get_tricode_by_fullname, get_fullname_by_tricode

In [65]:
def process_team_shots_and_opponent(input_date: str, team: str, db_config: Dict[str, str], model, poly) -> Dict[str, Any]:
    """
    Processes a team's lineup shots and retrieves opponent's on-ice statistics for a given date.
    
    This function performs the following steps:
        1. Runs `process_team_lineup_shots` for the specified team and date.
        2. Retrieves the game_id where the team played on the specified date.
        3. Identifies the opponent team based on the game_id.
        4. Converts the opponent team's triCode to its full name.
        5. Uses `nst_team_on_ice_scraper` to fetch the opponent team's on-ice statistics.
    
    Args:
        input_date (str): The reference date in 'YYYY-MM-DD' format.
        team (str): The three-letter team code (e.g., 'TOR').
        db_config (Dict[str, str]): Database configuration dictionary.
        model: Loaded regression model.
        poly: Loaded PolynomialFeatures transformer.
    
    Returns:
        Dict[str, Any]: A dictionary containing the team's lineup shots DataFrame, game_id, opponent team code,
                        and opponent's on-ice statistics DataFrame.
    
    Raises:
        ValueError: If the team did not play on the specified date or if the opponent cannot be determined.
    """
    try:
        # Step 1: Determine the reference date, if input_date is None, use yesterday's date
        if input_date is None:
            today_datetime = datetime.now()
            yesterday_datetime = today_datetime - timedelta(days=1, hours=6)  # Adjust for UTC offset if necessary
            input_date = yesterday_datetime.strftime('%Y-%m-%d')
        else:
            input_datetime = datetime.strptime(input_date, '%Y-%m-%d')
            day_before_input_dt = input_datetime - timedelta(days=1, hours=6)
            day_before_input = day_before_input_dt.strftime('%Y-%m-%d')

        # Step 2: Run process_team_lineup_shots for the given team and date
        team_shots_df = process_team_lineup_shots(input_date, team, db_config)
        print(f"Processed lineup shots for team {team} on {input_date}.")

        # Step 3: Retrieve matchup games for the input date
        print(f"Retrieving matchup games for date {input_date}.")
        temp_data = get_matchup_games(input_date, input_date)
        game_ids = temp_data.get('game_ids', {}).get('id', [])
        game_dates = temp_data.get('game_ids', {}).get('date', [])

        # Initialize variables
        game_id = None
        opponent_team_tricode = None

        # Step 4: Identify the game_id involving the specified team
        for gid, gdate in zip(game_ids, game_dates):
            print(f"Checking Game ID: {gid} on Date: {gdate}")
            boxscore = get_game_boxscore(gid, clean=True)
            away_team = boxscore.get('away_team')
            home_team = boxscore.get('home_team')
            print(f"Away Team: {away_team}, Home Team: {home_team}")
            
            if team.upper() == away_team.upper():
                opponent_team_tricode = home_team.upper()
                game_id = gid
                print(f"Team {team} found as Away Team in Game ID {gid}. Opponent TriCode: {opponent_team_tricode}")
                break
            elif team.upper() == home_team.upper():
                opponent_team_tricode = away_team.upper()
                game_id = gid
                print(f"Team {team} found as Home Team in Game ID {gid}. Opponent TriCode: {opponent_team_tricode}")
                break

        if not game_id or not opponent_team_tricode:
            raise ValueError(f"Team {team} did not play on {input_date} or could not determine opponent.")

        # Step 5: Convert opponent_team_tricode to full name
        opponent_team_fullname = get_fullname_by_tricode(opponent_team_tricode)
        if opponent_team_fullname is None:
            raise ValueError(f"Could not find full name for opponent TriCode '{opponent_team_tricode}'.")
        print(f"Opponent Team Full Name: {opponent_team_fullname}")

        # Step 6: Use nst_team_on_ice_scraper to fetch opponent's on-ice statistics
        print(f"Fetching on-ice statistics for opponent team {opponent_team_fullname}.")
        team_stats_df = nst_team_on_ice_scraper(
            fromseason=20242025,
            thruseason=20242025,
            startdate='',
            enddate=day_before_input,
            stype=2,
            sit='all'
        )
        
        # Assuming 'team' is the column name for team full names in the scraped DataFrame
        if 'team' not in team_stats_df.columns:
            raise KeyError("Column 'team' not found in opponent_stats_df. Please verify the scraped data.")
        
        team_stats_df = team_stats_df[team_stats_df['team'].str.lower() == opponent_team_fullname.lower()]
        if team_stats_df.empty:
            raise ValueError(f"No on-ice statistics found for opponent team '{opponent_team_fullname}'.")
        print(f"Retrieved on-ice statistics for opponent team {opponent_team_fullname}.")

        return {
            'team_shots': team_shots_df,
            'game_id': game_id,
            'opponent_team': opponent_team_fullname,
            'opponent_stats': team_stats_df
        }

    except Exception as e:
        print(f"An error occurred during processing: {e}")
        raise

In [66]:
# Example usage of the process_team_and_opponent function
input_date = '2024-12-05'
team = 'FLA'  # Example team code
matchup_data = process_team_shots_and_opponent(input_date, team, db_config, loaded_model, loaded_poly)

# Accessing the data
team_shots = matchup_data['team_shots']
game_id = matchup_data['game_id']
opponent_team = matchup_data['opponent_team']
opponent_stats = matchup_data['opponent_stats']

# Displaying the results
print(f"Game ID: {game_id}")
print(f"Opponent Team: {opponent_team}")
print("Team Shots:")
print(team_shots)
print("Opponent On-Ice Statistics:")
print(opponent_stats)

Fetching data for reference date: 2024-12-04
Player and goalie statistics fetched successfully.
Game ID: 2024020399
Added player 'Sam Bennett' to Forwards slot 1.
Added player 'A.J. Greer' to Forwards slot 2.
Added player 'Jonah Gadjovich' to Forwards slot 3.
Added player 'Sam Reinhart' to Forwards slot 4.
Added player 'Anton Lundell' to Forwards slot 5.
Added player 'Aleksander Barkov' to Forwards slot 6.
Added player 'Evan Rodrigues' to Forwards slot 7.
Added player 'Matthew Tkachuk' to Forwards slot 8.
Added player 'Carter Verhaeghe' to Forwards slot 9.
Added player 'Eetu Luostarinen' to Forwards slot 10.
Added player 'Jesper Boqvist' to Forwards slot 11.
Added player 'Tomas Nosek' to Forwards slot 12.
Added player 'Aaron Ekblad' to Defense slot 1.
Added player 'Dmitry Kulikov' to Defense slot 2.
Added player 'Adam Boqvist' to Defense slot 3.
Added player 'Gustav Forsling' to Defense slot 4.
Added player 'Niko Mikkola' to Defense slot 5.
Added player 'Nate Schmidt' to Defense slot 6